In [1]:
%%capture
!pip install --upgrade tensorflow_hub
# !pip install -U -huggingface_hub

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../../')
from eval_utils import *
sys.path.pop()

2023-08-30 19:08:39.951031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 19:08:41.200646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')
dataset_val = input('Enter the number of samples to run on (100 or 776): ')
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')
cand_size = int(input('enter number of candidates (recommended 12 for quicker run, 50 otherwise): '))

imdb_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
imdb_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
imdb_model.to(device)
imdb_pipeline = pipeline('sentiment-analysis', model=imdb_model, tokenizer=imdb_tokenizer)
imdb_pipeline.device = next(imdb_model.parameters()).device

imdb_model_directory = "../../../../models/bert-uncased_maskedlm_imdb_july31_chk3"
finetuned_imdb_maskedlm = BertForMaskedLM.from_pretrained(imdb_model_directory)
finetuned_imdb_maskedlm.to(device)
imdb_fill_mask = pipeline("fill-mask", model=finetuned_imdb_maskedlm, tokenizer=imdb_tokenizer)
imdb_fill_mask.device = next(imdb_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = TextFoolerJin2019

if dataset_val == '100':
    loaded_imdb_100 = Dataset.load_from_disk('../../data/filtered_imdb_clean_100')
    imdb_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_imdb_100))
    dataset = imdb_100
    dataset_name = 'imdb100'
elif dataset_val =='776':
    # because there were problems running epxeiremtns for days on end with the larger dataset,
    # the 776 samples were split up into 250, 250, 276 (sections 1,2,3 respectively).
    # hence experiments are run on each section separately, with final score coming from the collective results.
    dataset_section = input('Which section of the dataset would you like to load and test on? '
                            '(integer between 0 and 7) : ')
    loaded_imdb_776 = Dataset.load_from_disk(f'../../data/filtered_imdb_clean_776_into100_{dataset_section}')
    imdb_776 = textattack.datasets.Dataset(convert_to_tuples(loaded_imdb_776))
    dataset = imdb_776
    dataset_name = f'imdb776_into100_{dataset_section}'
else:
    raise ValueError('Number of samples not supported')
    
if defense == "default":
    imdb_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(imdb_model, imdb_tokenizer)
elif defense == "logit":
    imdb_wrapper = MaskDemaskWrapper(imdb_model, imdb_tokenizer, imdb_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    imdb_wrapper = MaskDemaskWrapper(imdb_model, imdb_tokenizer, imdb_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    imdb_wrapper = MaskDemaskWrapper(imdb_model, imdb_tokenizer, imdb_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(imdb_wrapper)

# change candidate size
attack.transformation.max_candidates = cand_size
# adjust attack threshold to match Li et al. 2023 (0.7 theshold for imdb Universal sentences encoder):
attack.constraints[2] = UniversalSentenceEncoder(metric = 'angular', threshold = 0.7, 
                                                 window_size = 15, skip_text_shorter_than_window=True, 
                                                 compare_against_original=False)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:0
Enter the number of samples to run on (100 or 776): 776
Specify a defense type among "default", "logit", "maj_log", "one_hot": one_hot
enter number of candidates (recommended 12 for quicker run, 50 otherwise): 50
Which section of the dataset would you like to load and test on? (integer between 0 and 7) : 7
using num_voter = 11 and mask_pct = 0.3 with dataset = imdb776_into100_7...
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (

  0%|          | 0/76 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2023-08-30 19:10:43.094145: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-30 19:10:45.187713: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]
[Succeeded / Failed / Skipped / Total] 1 /

In [ ]:
13